# Using Partitioning to Optimize a Circuit

Synthesis is a very powerful circuit optimization technique. However, the input size to even QFAST doesn't scale to larger circuits well. In fact, to be able to synthesize a circuit currently, we will need to be able to simulate it. This will ultimately cap the scaling of synthesis algorithms. However, we can still use a synthesis tool together with partitioner to optimize small blocks of a circuit at a time. BQSKit was designed for this exact use case and in this guide, we will explore how to accomplish this. It is recommended that you read the other tutorials first.

In [ ]:
# Load a 16-qubit time evolution circuit generated from the ArQTIC circuit generator.
from bqskit.ir import Circuit

circuit = Circuit.from_file('tfim-16-20.qasm')

for gate in circuit.gate_set:
    print(f"{gate} Count:", circuit.count(gate))

We will partition the circuit and then use the `ForEachBlockPass` to perform operations on the individual blocks. Note the `ForEachBlockPass` will run the sub tasks in parallel using dask.

In [ ]:
from bqskit.compiler import CompilationTask
from bqskit.compiler import Compiler
from bqskit.passes import QuickPartitioner
from bqskit.passes import ForEachBlockPass
from bqskit.passes import QSearchSynthesisPass
from bqskit.passes import ScanningGateRemovalPass
from bqskit.passes import UnfoldPass

task = CompilationTask(circuit, [
    QuickPartitioner(3),
    ForEachBlockPass([QSearchSynthesisPass(), ScanningGateRemovalPass()]),
    UnfoldPass(),
])

# Finally, we construct a compiler and submit the task
with Compiler() as compiler:
    synthesized_circuit = compiler.compile(task)
    
for gate in synthesized_circuit.gate_set:
    print(f"{gate} Count:", synthesized_circuit.count(gate))

## Gatesets

Just like we changed the gates used by QSearch in the Search Synthesis tutorial, we can change the gates for the entire circuit using the same method.

**Exercise:** Change the gates used in the below example to change the gate set for the circuit.

In [ ]:
from bqskit.ir.gates import ISwapGate, U3Gate
from bqskit.passes.search import SimpleLayerGenerator

layer_gen = SimpleLayerGenerator(two_qudit_gate=ISwapGate(), single_qudit_gate_1=U3Gate())

configured_qsearch_pass = QSearchSynthesisPass(layer_generator=layer_gen)

task = CompilationTask(circuit, [
    QuickPartitioner(3),
    ForEachBlockPass([configured_qsearch_pass, ScanningGateRemovalPass()]),
    UnfoldPass(),
])

with Compiler() as compiler:
    synthesized_circuit = compiler.compile(task)
    
for gate in synthesized_circuit.gate_set:
    print(f"{gate} Count:", synthesized_circuit.count(gate))

## Block Size

Increasing the partitioner's block size will likely lead to better results at a runtime cost. If you have the computing resources, you can launch a Dask cluster and connect to it via `Compiler()`. The ForEachBlockPass will efficiently distribute the work.

In [ ]:
from bqskit.passes import OptimizedLEAPPass


task = CompilationTask(circuit, [
    QuickPartitioner(4),
    ForEachBlockPass([OptimizedLEAPPass]),
    UnfoldPass(),
])

with Compiler() as compiler:
    synthesized_circuit = compiler.compile(task)
    
for gate in synthesized_circuit.gate_set:
    print(f"{gate} Count:", synthesized_circuit.count(gate))